In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import torch.nn as nn
import torch
import sklearn.preprocessing as preprocessing
import os

from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader


import warnings
warnings.filterwarnings(action='ignore')
from copy import deepcopy

from Prescriptor import Prescriptor
from Evolution.crossover import UniformCrossover, WeightedSumCrossover
from Evolution.mutation import MultiplyNormalMutation, MultiplyUniformMutation, AddNormalMutation, AddUniformMutation, ChainMutation, FlipSignMutation
from Evolution.selection import RouletteSelection, TournamentSelection, ParetoLexsortSelection
from Evolution import Evolution

In [3]:
# train.py

import os
import torch
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

from data_loader import load_data_1m
from feature_calculations import (
    resample_data, calculate_MA_data, calculate_ema_bollinger_bands, calculate_rsi,
    calculate_macd, calculate_stochastic_oscillator, calculate_adx, calculate_atr,
    calculate_obv, calculate_williams_r, base_feature_fn, cyclic_encode_fn, log_transform
)
from strategies import BB_fitness_fn
from dataset import make_dataset, replace_nan_with_zero
from train_functions_bi_cul import inference, fitness_fn, generation_valid, generation_test, calculate_fitness

from Prescriptor import Prescriptor
from Evolution.crossover import UniformCrossover, WeightedSumCrossover
from Evolution.mutation import MultiplyNormalMutation, MultiplyUniformMutation, AddNormalMutation, AddUniformMutation, ChainMutation, FlipSignMutation
from Evolution.selection import RouletteSelection, TournamentSelection, ParetoLexsortSelection
from Evolution import Evolution




In [4]:
state_dict_path = '/root/daily/bit_5/generation/generation_49.pt'
if os.path.exists(state_dict_path):
    state_dict = torch.load(state_dict_path)
    start_gen_idx = state_dict['generation'] + 1
    best_profit = state_dict['best_profit']
    best_chromosomes = state_dict['best_chromosomes']
    # prescriptor.load_state_dict(state_dict['prescriptor_state_dict'],strict=True)

In [5]:
best_profit

tensor([[-1.0000e+09, -1.0000e+09, -1.0000e+09,  1.0000e+09, -1.0000e+09],
        [-1.0000e+09, -1.0000e+09, -1.0000e+09,  1.0000e+09, -1.0000e+09],
        [-1.0000e+09, -1.0000e+09, -1.0000e+09,  1.0000e+09, -1.0000e+09],
        [-1.0000e+09, -1.0000e+09, -1.0000e+09,  1.0000e+09, -1.0000e+09],
        [-1.0000e+09, -1.0000e+09, -1.0000e+09,  1.0000e+09, -1.0000e+09],
        [-1.0000e+09, -1.0000e+09, -1.0000e+09,  1.0000e+09, -1.0000e+09],
        [-1.4313e+00,  5.0607e-01,  4.9333e-01,  1.2430e+02,  2.8206e-01],
        [-1.0000e+09, -1.0000e+09, -1.0000e+09,  1.0000e+09, -1.0000e+09],
        [-1.0000e+09, -1.0000e+09, -1.0000e+09,  1.0000e+09, -1.0000e+09],
        [-1.0000e+09, -1.0000e+09, -1.0000e+09,  1.0000e+09, -1.0000e+09],
        [-1.0000e+09, -1.0000e+09, -1.0000e+09,  1.0000e+09, -1.0000e+09],
        [-1.0000e+09, -1.0000e+09, -1.0000e+09,  1.0000e+09, -1.0000e+09]])

In [4]:
optimal_solution = best_profit[(best_profit[:, 4] > 0.8) & (best_profit[:, 3] <= 40)]
optimal_index = torch.where((best_profit[:, 4] > 0.8) & (best_profit[:, 3] <= 40))[0]

In [5]:
optimal_index

tensor([], dtype=torch.int64)

In [29]:
best_profit[optimal_index][:, 4]

tensor([1.7716, 3.1858, 0.9203, 0.9005, 0.8122, 1.1523, 0.8795, 0.8505, 1.2092,
        0.9597, 0.9444, 1.1975, 1.6272, 0.8660, 2.1316, 0.9200, 1.0717, 1.6610,
        1.5866, 1.0020, 3.2361])

In [30]:
fitness = calculate_fitness(deepcopy(optimal_solution.numpy()))
fitness_index = np.argsort(-fitness)

In [32]:
index = 0

In [33]:
optimal_solution[fitness_index][:, 0][index]

tensor(3.0549)

In [34]:
optimal_solution[fitness_index][:, 1][index]

tensor(2.5397)

In [35]:
optimal_solution[fitness_index][:, 2][index]

tensor(0.5400)

In [36]:
optimal_solution[fitness_index][:, 3][index]

tensor(27.6071)

In [37]:
optimal_solution[fitness_index][:, 4][index]

tensor(3.2361)